In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kg20c-scholarly-knowledge-graph/valid.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/all_relation_info.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/all_entity_info.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/README.md
/kaggle/input/kg20c-scholarly-knowledge-graph/test.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/train.txt


In [62]:
ROOT = '/kaggle/input/kg20c-scholarly-knowledge-graph/'

def get_txt_pd(columns, file_name):
    main = pd.read_csv(ROOT+file_name, sep='\t', header=None)
    main.columns = columns
    main = main.iloc[1:]
    return main

entity_ref = get_txt_pd(['id', 'name', 'type'], 'all_entity_info.txt')
train_data = get_txt_pd(['paper1', 'relation', 'paper2'], 'train.txt')
test_data = get_txt_pd(['paper1', 'relation', 'paper2'], 'test.txt')
valid_data = get_txt_pd(['paper1', 'relation', 'paper2'], 'valid.txt')

with open(ROOT+'all_relation_info.txt', 'r') as rel:
    data = rel.readlines()
    rem_new = lambda x: x.replace('\n', '')
    refs = [rem_new(x) for x in data]

vals = [i for i in range(len(refs))]
relation_ref = dict(zip(refs, vals))

relation_ref['self'] = 6

print(relation_ref)
all_nodes = list(set(entity_ref['id']))

np_table = np.identity(len(all_nodes)) * relation_ref['self']

{'id': 0, 'author_in_affiliation': 1, 'author_write_paper': 2, 'paper_cite_paper': 3, 'paper_in_domain': 4, 'paper_in_venue': 5, 'self': 6}


In [57]:
adjacency = pd.DataFrame(np_table, columns=all_nodes, index=all_nodes)

train_data['relation'] = train_data['relation'].replace(relation_ref)
train_list = train_data.values.tolist()
print(train_list[0])

for relation in train_list:
    adjacency[relation[0]][relation[2]] = relation[1]


['062B7298', 1, '00D4F325']


In [58]:
n_table = adjacency.to_numpy()
print(np.count_nonzero(n_table == 3))

adjacency.head()

adjacency.to_csv('./adjacency_dba.csv')
json_object = json.dumps(relation_ref)

with open('./ref_edges.json', 'w') as file:
    file.write(json_object)

7382


In [6]:
!pip3 install paperscraper

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.0/986.0 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 27.3 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=000be604dc1ce5a3ccc4cb954dd838e8cd21dfec3c425084503e85be2cf37e04
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=1f007d68d199b42800aacde

In [40]:
# from paperscraper import PaperScraper
# scraper = PaperScraper()

# !pip install google-search-results

from bs4 import BeautifulSoup
import requests
import json
import bs4 as bs 
import urllib.request 
from urllib.request import Request, urlopen 
import re
from serpapi import GoogleSearch

#id, authors, year, publication value/journal, abstract, title, id_ref/citations

all_titles = entity_ref[['id', 'name']]
all_titles['url'], all_titles['abstract'], all_titles['citations'], all_titles['publication value'], all_titles['year'], all_titles['authors'], all_titles['doi'] = '', '', '', '', '', '', ''

proxies = {
"http": 'http://209.50.52.162:9050', 
"https": 'http://209.50.52.162:9050'
}

all_names = all_titles['name'].tolist()

all_titles.head()

skipped = []

,id,name,url,abstract,citations,publication value,year,authors,doi
1,7C7CAEED,On rank correlation in information retrieval e...,,,,,,,
2,7AEE29E3,The Voting Model for People Search,,,,,,,
3,7D68490B,Document clustering with committees,,,,,,,
4,7A488256,A comparison of indexing techniques for Japane...,,,,,,,
5,7D5CD2DF,Feature selection for ranking,,,,,,,


In [47]:


for idx in range(197, len(all_names)):
    if idx == 297:
        break
    else:

        query = '+'.join(all_names[idx].split(' '))
        url_template = f"https://serpapi.com/search.json?engine=google_scholar&q={query}"

        params = {
          "engine": "google_scholar",
          "q": query,
          "hl": "en",
          "api_key": "2d2db50123cce476de21fc2f95080a71891d08a3edd179188040e2d2b2f66f00"
        }

        # "c9fd1f9e1aabf01316abde7496afa68c38493d0a9bd94b7739fa52e0b837e460"
        # "5b0f4c0c3cf5c913e84563a15a0cb5cbe1221c228aebf7b8def7a61df5eb61f9"

        search = GoogleSearch(params)
        results = search.get_dict()

        scrape_link = results['organic_results'][0]['link']
        print(scrape_link)

        all_titles['url'].iloc[idx] = scrape_link

        page = requests.get(scrape_link)
        soup = BeautifulSoup(page.content, "html.parser")

        if 'https://dl.acm.org/doi/abs/' in scrape_link:
            results = soup.find('div', class_='abstractSection abstractInFull').text
            author = soup.find('span', class_='loa__author-name').text
            publication_value = soup.find('div', class_="issue-item__detail").text
            date = soup.find('span', class_='CitationCoverDate').text
            doi_str = scrape_link
            doi_str = doi_str.replace('https://dl.acm.org/doi/abs/', '')
            citations_page = requests.get(f'https://dl.acm.org/action/ajaxShowCitedBy?doi={doi_str}')
            cit_soup = BeautifulSoup(citations_page.content, "html.parser")
            citations = cit_soup.find_all('span', class_='doi')
            citations = str(citations)
            for ch in ['<p', '<', '>', '[', ']', 'class=', '/p', 'ep_field_para', '''""''', 'span', 'doi']:
                if ch in citations:
                    citations = citations.replace(ch,'')

            all_titles['doi'].iloc[idx] = doi_str
            all_titles['abstract'].iloc[idx] = results
            all_titles['authors'].iloc[idx] = author
            all_titles['publication value'].iloc[idx] = publication_value
            all_titles['citations'].iloc[idx] = citations

        else:
            skipped.append(idx)
            pass

# elif 'https://theses.gla.ac.uk/'in scrape_link:
#     results = soup.find_all('p', class_='ep_field_para')
#     results = str(results)
#     for ch in ['<p', '<', '>', '[', ']', 'class=', '/p', 'ep_field_para', '''""''']:
#         if ch in results:
#             results = results.replace(ch,"")

# all_titles['abstract'].iloc[idx] = results
# print(idx)
#     except:
#         print('Not Compatible Sorry')

https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/345508.345578
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1571941.1572005
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1390334.1390393
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1277741.1277843
https://serpapi.com/search
https://link.springer.com/chapter/10.1007/978-1-4471-2099-5_27
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1008992.1009029
https://serpapi.com/search
https://dl.acm.org/doi/pdf/10.1145/312624.312669
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1277741.1277832
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1390334.1390354
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1277741.1277823
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/1277741.1277763
https://serpapi.com/search
https://dl.acm.org/doi/abs/10.1145/860435.860486
https://serpapi.com/search


KeyError: 'organic_results'

In [53]:
check_len = all_titles['publication value']
common_metadataset = all_titles.iloc[:len([x for x in check_len if len(x) != 0])+1]
common_metadataset.head()

,id,name,url,abstract,citations,publication value,year,authors,doi
1,7C7CAEED,On rank correlation in information retrieval e...,https://dl.acm.org/doi/abs/10.1145/1273221.127...,Some methods for rank correlation in evaluatio...,"""""10.1109/ICPR48806.2021.9411963/, """"10.1109...",ACM SIGIR ForumVolume 41Issue 1June 2007 pp ...,,Massimo Melucci,10.1145/1273221.1273223
2,7AEE29E3,The Voting Model for People Search,https://theses.gla.ac.uk/609/,,,,,,
3,7D68490B,Document clustering with committees,https://dl.acm.org/doi/abs/10.1145/564376.564412,\nDocument clustering is useful in many inform...,"""""10.1109/ACIT53391.2021.9677217/, """"10.4018...",SIGIR '02: Proceedings of the 25th annual inte...,,Patrick Pantel,10.1145/564376.564412
4,7A488256,A comparison of indexing techniques for Japane...,https://dl.acm.org/doi/abs/10.1145/160688.160728,\nA series of Japanese full-text retrieval exp...,"""""10.1109/SNLP.2009.5340946/, """"10.1007/s105...",SIGIR '93: Proceedings of the 16th annual inte...,,Hideo Fujii,10.1145/160688.160728
5,7D5CD2DF,Feature selection for ranking,https://dl.acm.org/doi/abs/10.1145/1277741.127...,\nRanking is a very important topic in informa...,"""""10.1007/978-3-030-95391-1_29/, """"10.1007/9...",SIGIR '07: Proceedings of the 30th annual inte...,,Xiubo Geng,10.1145/1277741.1277811


In [54]:
common_metadataset.to_csv('./common_metadataset_a.csv')

In [59]:
with open('./common_metadataset_a.csv', 'r') as file:
    df = pd.read_csv(file)

df.head()

,Unnamed: 0,id,name,url,abstract,citations,publication value,year,authors,doi
0,1,7C7CAEED,On rank correlation in information retrieval e...,https://dl.acm.org/doi/abs/10.1145/1273221.127...,Some methods for rank correlation in evaluatio...,"""""10.1109/ICPR48806.2021.9411963/, """"10.1109...",ACM SIGIR ForumVolume 41Issue 1June 2007 pp ...,NaN,Massimo Melucci,10.1145/1273221.1273223
1,2,7AEE29E3,The Voting Model for People Search,https://theses.gla.ac.uk/609/,NaN,NaN,NaN,NaN,NaN,NaN
2,3,7D68490B,Document clustering with committees,https://dl.acm.org/doi/abs/10.1145/564376.564412,\nDocument clustering is useful in many inform...,"""""10.1109/ACIT53391.2021.9677217/, """"10.4018...",SIGIR '02: Proceedings of the 25th annual inte...,NaN,Patrick Pantel,10.1145/564376.564412
3,4,7A488256,A comparison of indexing techniques for Japane...,https://dl.acm.org/doi/abs/10.1145/160688.160728,\nA series of Japanese full-text retrieval exp...,"""""10.1109/SNLP.2009.5340946/, """"10.1007/s105...",SIGIR '93: Proceedings of the 16th annual inte...,NaN,Hideo Fujii,10.1145/160688.160728
4,5,7D5CD2DF,Feature selection for ranking,https://dl.acm.org/doi/abs/10.1145/1277741.127...,\nRanking is a very important topic in informa...,"""""10.1007/978-3-030-95391-1_29/, """"10.1007/9...",SIGIR '07: Proceedings of the 30th annual inte...,NaN,Xiubo Geng,10.1145/1277741.1277811


In [72]:
all_nodes = df['id'].values.tolist()

np_adjacency = np.identity(len(all_nodes)) * 1
adjacency = pd.DataFrame(np_adjacency, columns=all_nodes, index=all_nodes)
adjacency.head()

node_1 = train_data['paper1'].values.tolist()
node_2 = train_data['paper2'].values.tolist()

for idx, node in enumerate(node_1):
    try:
        index = all_nodes.index(node_2[idx])
        adjacency[node].iloc[index] = float(1)
    except:
        pass

adjacency.head()
    

,7C7CAEED,7AEE29E3,7D68490B,7A488256,7D5CD2DF,7646C4F7,7CF9B0DC,78D3F0A3,7C55ADC9,7F9C2FCD,...,7FA844BE,7EF01968,7DD1A5C9,79CA2BA6,7DC501D5,799D1559,758CBA13,75349D25,78118BFA,76A00DCE
7C7CAEED,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7AEE29E3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7D68490B,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7A488256,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7D5CD2DF,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
np_adjacency = np.asarray(adjacency)
count = np.count_nonzero(np_adjacency == 1)

254
